In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm 
import wandb
from torch.optim.lr_scheduler import CosineAnnealingLR
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path, set_seeds
from src.utils.constants import Constants 
from src.datasets.eeg_dataset import EEGDataset
from src.models.gru import GRUModel

wandb.login()

2025-10-22 13:14:30,879 :: root :: INFO :: Initialising Utils
2025-10-22 13:14:30,881 :: root :: INFO :: Initialising Datasets
2025-10-22 13:14:30,903 :: root :: INFO :: Initialising Models
wandb: Currently logged in as: maikotrede (hms-hslu-aicomp-hs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
class CFG:
    seed = 42
    n_splits = 5
    data_path = '../../../data/'

    model_name = 'GRU'
    hidden_units = 128
    num_layers = 2
    target_size = 6 
    
    sampling_rate = 200 # Hz
    sequence_duration = 50 

    num_channels = 20 
    
    batch_size = 32
    num_workers = 0
    epochs = 15
    lr = 1e-4

CFG.sequence_length = CFG.sequence_duration * CFG.sampling_rate 

set_seeds(CFG.seed)

TARGETS = Constants.TARGETS

In [ ]:
def get_dataloaders(df, fold_id):
    train_df = df[df['fold'] != fold_id].reset_index(drop=True)
    valid_df = df[df['fold'] == fold_id].reset_index(drop=True)

    train_dataset = EEGDataset(df=train_df, data_path=CFG.data_path, mode='train', downsample_factor=3)

    valid_dataset = EEGDataset(df=valid_df, data_path=CFG.data_path, mode='valid', downsample_factor=3)

    train_loader = DataLoader(
        train_dataset, batch_size=CFG.batch_size, shuffle=True,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=True
    )
    
    valid_loader = DataLoader(
        valid_dataset, batch_size=CFG.batch_size, shuffle=False,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=False
    )
    
    return train_loader, valid_loader

In [4]:
def run_training(df, DATA_PREPARATION_VOTE_METHOD):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    all_oof = []
    all_true = []
    
    fold_scores = []

    for fold in range(CFG.n_splits):
        print(f"\n========== FOLD {fold} ==========")

        config = {
            "architecture": CFG.model_name, "hidden_units": CFG.hidden_units, "num_layers": CFG.num_layers,
            "fold": fold, "features": "raw_eeg", "sequence_duration": f"{CFG.sequence_duration}s",
            "optimizer": "AdamW", "learning_rate": CFG.lr, "batch_size": CFG.batch_size,
            "epochs": CFG.epochs, "seed": CFG.seed, "Scheduler": "CosineAnnealingLR" 
        }

        wandb.init(
            project="hms-aicomp-gru",
            name=f"gru-raw-eeg-fold{fold}", 
            tags=['gru-baseline', f'fold{fold}'],
            config=config
        )

        model = GRUModel(
            input_size=CFG.num_channels,
            hidden_size=CFG.hidden_units,
            num_layers=CFG.num_layers,
            num_classes=CFG.target_size
        )
        model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.epochs)
        loss_fn = nn.KLDivLoss(reduction='batchmean')
        train_loader, valid_loader = get_dataloaders(df, fold)

        best_val_loss = float('inf')
        best_model_path = None

        for epoch in range(CFG.epochs):
            print(f"   --- Epoch {epoch+1}/{CFG.epochs} ---")
            
            model.train()
            train_loss = 0
            for signals, labels in tqdm(train_loader, desc="Training"):
                signals, labels = signals.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(signals)
                log_probs = F.log_softmax(outputs, dim=1)
                loss = loss_fn(log_probs, labels)
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                train_loss += loss.item() * signals.size(0)
                wandb.log({"train/loss": loss.item()})
            train_loss /= len(train_loader.dataset)

            model.eval()
            valid_loss = 0
            with torch.no_grad():
                for i, (signals, labels) in enumerate(tqdm(valid_loader, desc="Validation")):
                    signals, labels = signals.to(device), labels.to(device)
                    outputs = model(signals)
                    log_probs = F.log_softmax(outputs, dim=1)
                    loss = loss_fn(log_probs, labels)
                    valid_loss += loss.item() * signals.size(0)

            valid_loss /= len(valid_loader.dataset)
            
            epoch_lr = optimizer.param_groups[0]['lr']
            print(f"   Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Valid Loss = {valid_loss:.4f}, LR = {epoch_lr:.6f}")
            wandb.log({
                "epoch": epoch + 1, "train/epoch_loss": train_loss, "val/loss": valid_loss,
                "val/kl_div": valid_loss, "train/epoch_lr": epoch_lr
            })

            if valid_loss < best_val_loss:
                best_val_loss = valid_loss
                best_model_path = get_models_save_path() / "GRUModel" / DATA_PREPARATION_VOTE_METHOD / f'best_model_fold{fold}.pth'
                best_model_path.parent.mkdir(parents=True, exist_ok=True)
                torch.save(model.state_dict(), best_model_path)
                print(f"   New best model saved with validation loss: {best_val_loss:.4f}")
            scheduler.step()
        
        fold_scores.append(best_val_loss)
        wandb.summary['best_val_kl_div'] = best_val_loss

        if best_model_path:
            print(f"Loading best model from fold {fold} to generate OOF predictions...")
            model.load_state_dict(torch.load(best_model_path))
            model.eval()

            fold_oof_preds = []
            fold_oof_labels = []
            with torch.no_grad():
                for signals, labels in tqdm(valid_loader, desc=f"Generating OOF for Fold {fold}"):
                    signals = signals.to(device)
                    outputs = model(signals)
                    probs = F.softmax(outputs, dim=1).cpu().numpy()
                    fold_oof_preds.append(probs)
                    fold_oof_labels.append(labels.cpu().numpy())
            
            all_oof.extend(np.concatenate(fold_oof_preds))
            all_true.extend(np.concatenate(fold_oof_labels))

            artifact = wandb.Artifact(f'model-fold{fold}', type='model')
            artifact.add_file(best_model_path)
            wandb.log_artifact(artifact)
            print(f"\nLogged artifact for fold {fold} with best validation loss: {best_val_loss:.4f}")
        else:
            print("\nNo best model was saved during training for this fold.")
        
        wandb.finish()

    print("\nCalculating final OOF CV score from all collected predictions...")
    
    valid_indices = df[df['fold'].isin(range(CFG.n_splits))].index
    oof_df = pd.DataFrame(all_oof, index=valid_indices, columns=TARGETS)
    true_df = pd.DataFrame(all_true, index=valid_indices, columns=TARGETS)
    
    oof_df = oof_df.sort_index()
    true_df = true_df.sort_index()

    oof_tensor = torch.tensor(oof_df.values, dtype=torch.float32)
    true_tensor = torch.tensor(true_df.values, dtype=torch.float32)

    kl_loss = nn.KLDivLoss(reduction="batchmean")
    overall_cv_score = kl_loss(torch.log(oof_tensor), true_tensor).item()

    return overall_cv_score, fold_scores

DATA_PREPARATION_VOTE_METHOD = "sum_and_normalize"

print("Preparing data and creating folds...")
df = pd.read_csv(CFG.data_path + 'processed_data_sum_votes_window.csv') 

label_map = {t: i for i, t in enumerate(TARGETS)}
df['expert_consensus'] = df[TARGETS].idxmax(axis=1)

print('Train shape:', df.shape)
print('Targets', list(TARGETS))

fold_creator = KFoldCreator(n_splits=CFG.n_splits, seed=CFG.seed)
df = fold_creator.create_folds(df, stratify_col='expert_consensus', group_col='patient_id')

print("Folds created. Value counts per fold:")
print(df['fold'].value_counts())



Preparing data and creating folds...
Train shape: (17089, 12)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
Folds created. Value counts per fold:
fold
0    3741
1    3703
2    3527
4    3081
3    3037
Name: count, dtype: int64


In [5]:
if __name__ == '__main__':
    overall_cv_score, all_fold_scores = run_training(df, DATA_PREPARATION_VOTE_METHOD)
    print("\n" + "="*50)
    print("              CROSS-VALIDATION RESULTS")
    print("="*50)
    print(f"OOF KL Score (calculated across all folds): {overall_cv_score:.4f}")
    print(f"Mean of individual fold scores: {np.mean(all_fold_scores):.4f}")
    print("="*50)

Using device: cuda

========== FOLD 0 ==========


   --- Epoch 1/15 ---


Training:   0%|          | 0/417 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

   Epoch 1: Train Loss = 1.2492, Valid Loss = 1.3072, LR = 0.001000
   New best model saved with validation loss: 1.3072
   --- Epoch 2/15 ---


...

   Epoch 15: Train Loss = 0.8770, Valid Loss = 1.3870, LR = 0.000011
Loading best model from fold 4 to generate OOF predictions...


Generating OOF for Fold 4:   0%|          | 0/97 [00:00<?, ?it/s]


Logged artifact for fold 4 with best validation loss: 1.2484


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train/epoch_loss,██▇▇▆▆▅▅▄▃▂▂▁▁▁
train/epoch_lr,███▇▇▆▆▅▄▃▃▂▂▁▁
train/loss,█▅▆█▇▇▇▇▇▆▄▅▅▄▅▅▄▆▆▆▅▃▄▃▂▃▄▅▆▃▃▁▃▄▃▁▄▄▂▆
val/kl_div,▂▁▂▂▃▂▃▃▆▅▆▇▇██
val/loss,▂▁▂▂▃▂▃▃▆▅▆▇▇██
best_val_kl_div,1.24845
epoch,15
train/epoch_loss,0.87698
train/epoch_lr,1e-05
train/loss,0.86102



Calculating final OOF CV score from all collected predictions...

              CROSS-VALIDATION RESULTS
OOF KL Score (calculated across all folds): 1.1636
Mean of individual fold scores: 1.1662


In [6]:
print(f"OOF KL Score (calculated across all folds): {overall_cv_score:.4f}")
print(f"Mean of individual fold scores: {np.mean(all_fold_scores):.4f}")

OOF KL Score (calculated across all folds): 1.1636
Mean of individual fold scores: 1.1662
